# Installation
- Follow directions at the PySAL-ArcGIS-Toolbox Git Repository [https://github.com/Esri/PySAL-ArcGIS-Toolbox]
- Please make note of the section on **Adding a Git Project to your ArcGIS Installation Python Path**.


In [11]:
import arcpy as ARCPY
import arcgisscripting as ARC
import SSDataObject as SSDO
import SSUtilities as UTILS
import WeightsUtilities as WU
import numpy as NUM
import scipy as SCIPY
import pysal as PYSAL
import os as OS
import pandas as PANDAS

# Example: Testing the Income Convergence Hypothesis in California Counties (1969 - 2010)

- Use the Auto-Model Spatial Econometric Tool to identify the appropriate model

- Regressing the growth rate of incomes on the log of starting incomes 

    - a significant negative coefficient indicates convergence

- The percentage of the population w/o a high school education and the population itself are the other exogenous factors.

## Importing Your Data into a PANDAS DataFrame

In [12]:
inputFC = r'../data/CA_Polygons.shp'
fullFC = OS.path.abspath(inputFC)
fullPath, fcName = OS.path.split(fullFC)
ssdo = SSDO.SSDataObject(inputFC)
uniqueIDField = "MYID"
fieldNames = ['GROWTH', 'LOGPCR69', 'PERCNOHS', 'POP1969']
ssdo.obtainData(uniqueIDField, fieldNames)
df = ssdo.getDataFrame()
print(df.head())

       GROWTH  LOGPCR69  PERCNOHS  POP1969
158  0.011426  0.176233      37.0  1060099
159 -0.137376  0.214186      38.3      398
160 -0.188417  0.067722      41.4    11240
161 -0.085070 -0.118248      42.9   101057
162 -0.049022 -0.081377      48.1    13328


## Use the PySAL-ArcGIS Utilities to Read in Spatial Weights Files

In [9]:
import pysal2ArcUtils as PYSAL_UTILS
swmFile = OS.path.join(fullPath, "queen.swm")
W = PYSAL_UTILS.PAT_W(ssdo, swmFile)
w = W.w

kernelSWMFile = OS.path.join(fullPath, "knn8.swm")
KW = PYSAL_UTILS.PAT_W(ssdo, kernelSWMFile)
kw = KW.w

## Run the Auto Model Class and Export Your Data to an Output Feature Class

In [13]:
import AutoModel as AUTO
auto = AUTO.AutoSpace_PySAL(ssdo, "GROWTH", ['LOGPCR69', 'PERCNOHS', 'POP1969'],
                            W, KW, pValue = 0.1, useCombo = True)
ARCPY.env.overwriteOutput = True
outputFC = r'../data/pysal_automodel.shp'
auto.createOutput(outputFC)